
Step 1. [Download history price and clean data](/notebooks/forex/hawkeye/1_download.ipynb)

Step 2. [Use monte carlo simulate each stock's movement for 10000 times](/notebooks/forex/hawkeye/2_process.ipynb)

**Step 3. Apply some heuristic rules to give stock recommendation**

In [72]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

sys.path.append('/opt/hawkeye')
from asx import *

this_week=20190607
last_week=20190531
path = f'/opt/hawkeye/data/{this_week}/result.csv'
last_path = f'/opt/hawkeye/data/{last_week}/result.csv'

def week_return_avg(df):
    return round(df['week_return'].sum()/len(df),4)

df = pd.read_csv(path,
                 usecols=[
                     'code', 'last_date', 'start price', 'sim_mean',
                     'sim_diff', 'VaR 99%', 'VaR 99% Percent', 'volume_mean',
                     'return_mean', 'return_sigma'
                 ],
                 index_col='code')
last_df = pd.read_csv(last_path,
                      usecols=[
                          'code', 'last_date', 'start price', 'sim_mean',
                          'sim_diff', 'VaR 99%', 'VaR 99% Percent',
                          'volume_mean', 'return_mean', 'return_sigma'
                      ],
                      index_col='code')
df.head()
df['return'] = round(df['sim_diff'] / df['start price'] * 100, 3)
df['last_return'] = round(last_df['sim_diff'] / last_df['start price'] * 100,
                          3)
df['return_increase'] = df['return'] - df['last_return']
df['week_return'] = round(
    (df['start price'] - last_df['start price']) / df['start price'] * 100, 3)
df.drop(columns=['sim_diff', 'sim_mean', 'VaR 99%'], inplace=True)

print(f'Total stock number = {len(df)}')

print('70%% pricetile = %s' % df['start price'].quantile(0.7))
print('Sim return 90%% percentile = %s' % df['return'].quantile(0.9))

volume_threshold = df['volume_mean'].quantile(0.1)
print(f'Volume 90% percentile = {volume_threshold}')

df = df[(df['volume_mean'] > volume_threshold) & (df['start price'] > 0.1)]
print(len(df))
df.sort_values(by='return', ascending=False).head(20)

Total stock number = 1495
70% pricetile = 0.9389999999999998
Sim return 90% percentile = 1.1824000000000012
Volume 90% percentile = 20334.305050505056
781


,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
OCC,2019-06-07,0.405,57.5710,2.987168e+06,0.033110,0.365770,7.060,NaN,NaN,NaN
FTC,2019-06-07,0.145,34.8615,8.293592e+05,0.024464,0.190723,4.869,5.325,-0.456,-6.897
FEX,2019-06-07,0.115,16.4710,8.985051e+06,0.021486,0.093235,4.226,3.991,0.235,8.696
8CO,2019-06-07,0.160,12.7603,2.668630e+05,0.019369,0.075834,3.857,3.486,0.371,3.125
LTR,2019-06-07,0.105,16.7366,1.306565e+07,0.018831,0.094523,3.855,3.442,0.413,0.000
ANO,2019-06-07,6.480,11.7017,1.021522e+05,0.019508,0.068453,3.785,4.132,-0.347,-4.321
WZR,2019-06-07,0.135,18.0928,5.923985e+06,0.016248,0.101867,3.264,3.375,-0.111,-11.111
ISX,2019-06-07,0.650,9.1665,2.258699e+06,0.016403,0.056279,3.232,3.123,0.109,4.615
WKT,2019-06-07,0.370,8.7243,8.183202e+05,0.014655,0.052304,2.930,2.672,0.258,4.054


In [73]:
# high mean return 
high_return=df.sort_values(by='return', ascending=False).head(10)
print('High return last week performance avg. = %s%%'% week_return_avg(high_return))
high_return

High return last week performance avg. = -3.4192%


,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
OCC,2019-06-07,0.405,57.5710,2.987168e+06,0.033110,0.365770,7.060,NaN,NaN,NaN
FTC,2019-06-07,0.145,34.8615,8.293592e+05,0.024464,0.190723,4.869,5.325,-0.456,-6.897
FEX,2019-06-07,0.115,16.4710,8.985051e+06,0.021486,0.093235,4.226,3.991,0.235,8.696
8CO,2019-06-07,0.160,12.7603,2.668630e+05,0.019369,0.075834,3.857,3.486,0.371,3.125
LTR,2019-06-07,0.105,16.7366,1.306565e+07,0.018831,0.094523,3.855,3.442,0.413,0.000
ANO,2019-06-07,6.480,11.7017,1.021522e+05,0.019508,0.068453,3.785,4.132,-0.347,-4.321
WZR,2019-06-07,0.135,18.0928,5.923985e+06,0.016248,0.101867,3.264,3.375,-0.111,-11.111
ISX,2019-06-07,0.650,9.1665,2.258699e+06,0.016403,0.056279,3.232,3.123,0.109,4.615
WKT,2019-06-07,0.370,8.7243,8.183202e+05,0.014655,0.052304,2.930,2.672,0.258,4.054


In [74]:
# high price

high_price=df[df['start price']>1].sort_values(by='return', ascending=False).head(10)
print('High price last week performance avg. = %s%%'% week_return_avg(high_price))
high_price

High price last week performance avg. = -6.0008%


,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
ANO,2019-06-07,6.48,11.7017,1.021522e+05,0.019508,0.068453,3.785,4.132,-0.347,-4.321
DUB,2019-06-07,1.12,10.6083,6.550722e+05,0.012231,0.058187,2.318,2.720,-0.402,-10.714
Z1P,2019-06-07,3.03,7.8690,2.668193e+06,0.011510,0.045251,2.310,2.473,-0.163,-5.611
BUB,2019-06-07,1.08,10.1305,5.699048e+06,0.010815,0.056006,2.099,2.314,-0.215,-15.278
CIA,2019-06-07,2.93,7.3854,3.237166e+05,0.010259,0.040097,1.960,2.150,-0.190,-8.532
CRD,2019-06-07,1.72,7.8968,2.929931e+05,0.009677,0.045503,1.866,1.849,0.017,1.744
5GN,2019-06-07,1.17,8.7282,3.869308e+05,0.009048,0.045935,1.720,2.119,-0.399,-14.103
JIN,2019-06-07,17.25,4.9189,3.272871e+05,0.008538,0.028880,1.692,1.867,-0.175,-4.348
AD8,2019-06-07,7.76,3.9571,2.011405e+05,0.008159,0.025422,1.603,1.531,0.072,0.773


In [75]:
# highest price

highest_price=df[df['return']>0.8].sort_values(by='start price', ascending=False).head(10)
print('Highest price last week performance avg. = %s%%'% week_return_avg(highest_price))
highest_price

Highest price last week performance avg. = -1.4792%


,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
MFG,2019-06-07,45.83,2.5907,5.669468e+05,0.005556,0.016239,1.083,1.195,-0.112,5.237
APX,2019-06-07,26.71,6.0746,8.532511e+05,0.006893,0.032860,1.245,1.524,-0.279,-4.380
APT,2019-06-07,24.17,7.3986,2.140057e+06,0.006590,0.039193,1.286,1.276,0.010,0.538
JIN,2019-06-07,17.25,4.9189,3.272871e+05,0.008538,0.028880,1.692,1.867,-0.175,-4.348
BRG,2019-06-07,16.73,4.2420,3.204092e+05,0.004874,0.023434,0.933,0.953,-0.020,-1.913
IEL,2019-06-07,16.71,4.8991,7.103827e+05,0.004869,0.025849,0.964,1.017,-0.053,-2.274
LOV,2019-06-07,11.32,5.7355,3.075329e+05,0.006353,0.031493,1.223,1.244,-0.021,-0.707
APE,2019-06-07,10.00,4.0390,5.638414e+04,0.005350,0.022516,1.052,1.222,-0.170,-4.500
DLX,2019-06-07,9.74,5.5392,1.750063e+06,0.004328,0.028358,0.821,0.854,-0.033,0.103


In [76]:
# low risk
var_threshold = df['VaR 99% Percent'].quantile(0.3)
print(var_threshold)
low_risk = df[df['VaR 99% Percent'] < var_threshold].sort_values(by='return', ascending=False).head(10)
print('Low risk last week performance avg. = %s%%'% week_return_avg(low_risk))
low_risk

4.2959
Low risk last week performance avg. = -0.0168%


,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
AD8,2019-06-07,7.76,3.9571,2.011405e+05,0.008159,0.025422,1.603,1.531,0.072,0.773
DDR,2019-06-07,5.06,3.7204,1.553135e+05,0.006426,0.022400,1.245,1.297,-0.052,-1.581
MFG,2019-06-07,45.83,2.5907,5.669468e+05,0.005556,0.016239,1.083,1.195,-0.112,5.237
APE,2019-06-07,10.00,4.0390,5.638414e+04,0.005350,0.022516,1.052,1.222,-0.170,-4.500
BRG,2019-06-07,16.73,4.2420,3.204092e+05,0.004874,0.023434,0.933,0.953,-0.020,-1.913
SUL,2019-06-07,9.34,3.5701,1.093025e+06,0.004144,0.019399,0.799,0.667,0.132,0.964
NCK,2019-06-07,7.00,3.5445,2.022262e+05,0.003973,0.019558,0.798,0.747,0.051,-1.000
ASB,2019-06-07,2.85,3.3975,1.001346e+06,0.003989,0.018688,0.780,0.801,-0.021,-2.105
CSR,2019-06-07,3.93,3.4118,3.698921e+06,0.003811,0.017905,0.746,0.713,0.033,-1.527


In [77]:
# high volume
high_volume = df.sort_values(by='volume_mean', ascending=False).head(10)
print('Highest volume last week performance avg. = %s%%'% week_return_avg(high_volume))
high_volume

Highest volume last week performance avg. = 0.1564%


,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
TLS,2019-06-07,3.720,2.2853,2.772613e+07,0.002754,0.012430,0.521,0.495,0.026,1.882
FMG,2019-06-07,7.850,5.0763,1.980632e+07,0.006924,0.028223,1.370,1.377,-0.007,-2.548
S32,2019-06-07,3.240,3.2817,1.890647e+07,-0.000191,0.014739,-0.040,-0.002,-0.038,-2.469
MGR,2019-06-07,3.130,1.9208,1.416705e+07,0.003223,0.011173,0.621,0.601,0.020,3.195
HSO,2019-06-06,2.460,1.3076,1.365031e+07,0.000888,0.006591,0.178,0.202,-0.024,0.000
LTR,2019-06-07,0.105,16.7366,1.306565e+07,0.018831,0.094523,3.855,3.442,0.413,0.000
VCX,2019-06-07,2.610,2.4552,1.254472e+07,-0.000173,0.010703,-0.047,0.027,-0.074,1.149
SCG,2019-06-07,3.840,2.5765,1.246554e+07,-0.000327,0.010895,-0.070,0.001,-0.071,1.042
AWC,2019-06-07,2.410,3.5428,1.239917e+07,0.001057,0.016692,0.219,0.307,-0.088,1.660


In [78]:
# high return increase in last week
high_return_increase = df.sort_values(by='return_increase',
                                      ascending=False).head(10)
print('Highest return increase last week performance avg. = %s%%'% week_return_avg(high_return_increase))
high_return_increase

Highest return increase last week performance avg. = 17.8556%


,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
MMM,2019-06-07,0.740,16.0900,143557.989899,0.010251,0.083116,2.013,0.617,1.396,36.486
K2F,2019-06-07,0.160,13.9776,101416.020202,0.003968,0.067895,0.781,0.084,0.697,28.125
CPH,2019-06-07,0.550,10.5364,342596.686869,0.001286,0.048784,0.256,-0.426,0.682,23.636
NGI,2019-06-07,3.800,6.3175,388325.272727,0.003651,0.031550,0.684,0.080,0.604,0.263
VMT,2019-06-07,0.115,18.0295,527778.191919,0.011006,0.093670,2.372,1.803,0.569,13.913
AKG,2019-06-07,0.425,5.4357,24819.636364,0.002170,0.026630,0.443,-0.105,0.548,12.941
KSS,2019-06-07,0.265,8.9817,469344.818182,0.011606,0.051457,2.283,1.758,0.525,16.981
HFR,2019-06-07,0.935,9.3412,116558.292929,0.004013,0.046238,0.916,0.431,0.485,21.925
SGI,2019-06-07,0.105,8.3868,74030.111111,-0.003289,0.035932,-0.614,-1.076,0.462,14.286


In [79]:
# ASX 20
asx_20_list = get_asx_20_list()
asx_20=df[df.index.isin(asx_20_list)].sort_values(by='return', ascending=False)
print('ASX 20 last week performance avg. = %s%%'% week_return_avg(asx_20))
asx_20

ASX 20 last week performance avg. = 0.642%


,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
TLS,2019-06-07,3.72,2.2853,2.772613e+07,0.002754,0.012430,0.521,0.495,0.026,1.882
RIO,2019-06-07,98.00,2.9437,1.837498e+06,0.002429,0.014964,0.479,0.553,-0.074,-2.347
BXB,2019-06-07,12.73,1.8511,3.894025e+06,0.002111,0.009980,0.405,0.370,0.035,4.478
AMC,2019-06-06,15.77,1.5633,4.003851e+06,0.001996,0.008873,0.402,0.438,-0.036,-1.332
TCL,2019-06-07,14.26,1.4484,5.338606e+06,0.001962,0.007889,0.380,0.369,0.011,2.314
WES,2019-06-07,37.65,2.4023,2.517015e+06,0.001983,0.012165,0.368,0.343,0.025,1.594
BHP,2019-06-07,37.84,2.3111,6.664376e+06,0.001644,0.011666,0.323,0.237,0.086,0.370
ANZ,2019-06-07,28.37,2.9973,5.802228e+06,0.001462,0.014469,0.297,0.308,-0.011,1.269
CBA,2019-06-07,80.00,2.3350,3.127479e+06,0.001318,0.011475,0.249,0.247,0.002,1.862


In [81]:
# highest return in ASX 200
asx_200_list = get_asx_200_list()
asx_200=df[df.index.isin(asx_200_list)].sort_values(by='return', ascending=False).head(20)
print('Top 20 of ASX 200 last week performance avg. = %s%%'% week_return_avg(asx_200))
asx_200

Top 20 of ASX 200 last week performance avg. = -0.0939%


,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
FMG,2019-06-07,7.85,5.0763,1.980632e+07,0.006924,0.028223,1.370,1.377,-0.007,-2.548
APT,2019-06-07,24.17,7.3986,2.140057e+06,0.006590,0.039193,1.286,1.276,0.010,0.538
APX,2019-06-07,26.71,6.0746,8.532511e+05,0.006893,0.032860,1.245,1.524,-0.279,-4.380
LYC,2019-06-07,2.77,10.1446,6.718955e+06,0.006082,0.050779,1.186,1.564,-0.378,-10.108
MFG,2019-06-07,45.83,2.5907,5.669468e+05,0.005556,0.016239,1.083,1.195,-0.112,5.237
IEL,2019-06-07,16.71,4.8991,7.103827e+05,0.004869,0.025849,0.964,1.017,-0.053,-2.274
BRG,2019-06-07,16.73,4.2420,3.204092e+05,0.004874,0.023434,0.933,0.953,-0.020,-1.913
DHG,2019-06-07,3.24,6.7131,1.824723e+06,0.004492,0.033820,0.842,0.932,-0.090,-1.543
DLX,2019-06-07,9.74,5.5392,1.750063e+06,0.004328,0.028358,0.821,0.854,-0.033,0.103


In [84]:
# high last week return
df.sort_values(by='week_return', ascending=False).head(20).sort_values(by='return', ascending=False)

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
CV1,2019-06-07,0.145,9.8858,3.089637e+05,0.011898,0.055234,2.377,2.185,0.192,10.345
VMT,2019-06-07,0.115,18.0295,5.277782e+05,0.011006,0.093670,2.372,1.803,0.569,13.913
E2M,2019-06-07,0.215,12.9209,2.093969e+05,0.012415,0.069609,2.362,2.238,0.124,11.628
KSS,2019-06-07,0.265,8.9817,4.693448e+05,0.011606,0.051457,2.283,1.758,0.525,16.981
MMM,2019-06-07,0.740,16.0900,1.435580e+05,0.010251,0.083116,2.013,0.617,1.396,36.486
ARU,2019-06-07,0.105,12.4762,2.048072e+06,0.009523,0.066691,1.898,1.636,0.262,13.333
BCC,2019-06-07,0.310,14.5033,5.996728e+04,0.008017,0.073432,1.423,1.090,0.333,19.355
RED,2019-06-07,0.160,9.3100,2.364549e+06,0.006443,0.048649,1.280,1.008,0.272,12.500
HFR,2019-06-07,0.935,9.3412,1.165583e+05,0.004013,0.046238,0.916,0.431,0.485,21.925


## Thisk week recommendations

In [85]:
# recommendations
selection = [
    'FEX',  # high return & high risk
    'ANO',  # high return & high price & low sigma
    'AD8',  # high price & low sigma & low risk
    'MFG',  # high price & low sigma & low risk
    'CHC',  # low sigma & low risk & asx 200
    'JBH',  # low sigma & low risk & asx 200
    'CV1',  # best recent performance
]
df[df.index.isin(selection)].sort_values(by='return', ascending=False).head(10)

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
FEX,2019-06-07,0.115,16.4710,8.985051e+06,0.021486,0.093235,4.226,3.991,0.235,8.696
ANO,2019-06-07,6.480,11.7017,1.021522e+05,0.019508,0.068453,3.785,4.132,-0.347,-4.321
CV1,2019-06-07,0.145,9.8858,3.089637e+05,0.011898,0.055234,2.377,2.185,0.192,10.345
AD8,2019-06-07,7.760,3.9571,2.011405e+05,0.008159,0.025422,1.603,1.531,0.072,0.773
MFG,2019-06-07,45.830,2.5907,5.669468e+05,0.005556,0.016239,1.083,1.195,-0.112,5.237
CHC,2019-06-07,10.940,1.9351,1.759536e+06,0.003874,0.011683,0.743,0.680,0.063,5.484
JBH,2019-06-07,27.720,2.4354,7.491848e+05,0.003262,0.013540,0.655,0.646,0.009,-1.443


## Last week review

In [21]:
selection = [
    'AVH',  # high return & high risk
    'ANO',  # high return & high risk
    'AGH',  # high return & meduim risk
    'Z1P',  # medium price & risk
    'MFG',  # high price & low risk
    'AD8',  # low risk
    'FMG',  # high volume & low risk
]
df[df.index.isin(selection)].sort_values(by='return', ascending=False).head(10)

,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
ANO,2019-06-07,6.480,11.7017,1.021522e+05,0.019508,0.068453,3.785,4.132,-0.347,-4.321
AVH,2019-06-07,0.340,12.9361,8.729732e+06,0.014120,0.073468,2.774,3.937,-1.163,-32.353
Z1P,2019-06-07,3.030,7.8690,2.668193e+06,0.011510,0.045251,2.310,2.473,-0.163,-5.611
AGH,2019-06-07,0.655,8.5707,4.189650e+05,0.009319,0.046423,1.826,2.012,-0.186,-12.977
AD8,2019-06-07,7.760,3.9571,2.011405e+05,0.008159,0.025422,1.603,1.531,0.072,0.773
FMG,2019-06-07,7.850,5.0763,1.980632e+07,0.006924,0.028223,1.370,1.377,-0.007,-2.548
MFG,2019-06-07,45.830,2.5907,5.669468e+05,0.005556,0.016239,1.083,1.195,-0.112,5.237


In [22]:
week_return = round(sum(df[df.index.isin(selection)]['week_return']), 3)
print('Last week return:', week_return)

Last week return: -51.8


## Signle performance review

In [11]:
def inspect(code):
    print(f'http://h.luotao.net/data/{this_week}/pic/{code}_line.png')
    print(f'http://h.luotao.net/data/{this_week}/pic/{code}.png')
    
    return df[df.index==code]

In [23]:
inspect('AVH')

http://h.luotao.net/data/20190607/pic/AVH_line.png
http://h.luotao.net/data/20190607/pic/AVH.png


,last_date,start price,VaR 99% Percent,volume_mean,return_mean,return_sigma,return,last_return,return_increase,week_return
code,,,,,,,,,,
AVH,2019-06-07,0.34,12.9361,8.729732e+06,0.01412,0.073468,2.774,3.937,-1.163,-32.353
